In [ ]:
 !pip install -q kagglehub
 import kagglehub
 import os


In [ ]:

 full_path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")
 print("Dataset downloaded to:", full_path)


Dataset downloaded to: /kaggle/input/140k-real-and-fake-faces


In [ ]:
 import os

 # List files in the dataset directory
 for root, dirs, files in os.walk(full_path):
     for file in files:
         if file.endswith(".csv"):
             print("📄 Found CSV:", os.path.join(root, file))


📄 Found CSV: /kaggle/input/140k-real-and-fake-faces/valid.csv
📄 Found CSV: /kaggle/input/140k-real-and-fake-faces/train.csv
📄 Found CSV: /kaggle/input/140k-real-and-fake-faces/test.csv


In [ ]:


 import pandas as pd
 import os

 # Dataset base path
 base_path = "/kaggle/input/140k-real-and-fake-faces"
 image_root = os.path.join(base_path, "real_vs_fake", "real-vs-fake")  # 👈 Corrected image base folder

 # Load CSVs
 train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
 val_df = pd.read_csv(os.path.join(base_path, "valid.csv"))
 test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

 # ✅ Fix: Add correct full image path
 train_df['full_path'] = train_df['path'].apply(lambda x: os.path.join(image_root, x))

 val_df['full_path'] = val_df['path'].apply(lambda x: os.path.join(image_root, x))
 test_df['full_path'] = test_df['path'].apply(lambda x: os.path.join(image_root, x))

 # Confirm it worked
 print("Train:", len(train_df), "| Val:", len(val_df), "| Test:", len(test_df))
 print(train_df[['full_path', 'label_str']].head())



Train: 100000 | Val: 20000 | Test: 20000
                                           full_path label_str
0  /kaggle/input/140k-real-and-fake-faces/real_vs...      real
1  /kaggle/input/140k-real-and-fake-faces/real_vs...      real
2  /kaggle/input/140k-real-and-fake-faces/real_vs...      real
3  /kaggle/input/140k-real-and-fake-faces/real_vs...      real
4  /kaggle/input/140k-real-and-fake-faces/real_vs...      real


In [ ]:
 print("Train CSV columns:", train_df.columns)
 print("Validation CSV columns:", val_df.columns)
 print("Test CSV columns:", test_df.columns)


Train CSV columns: Index(['Unnamed: 0', 'original_path', 'id', 'label', 'label_str', 'path',
       'full_path'],
      dtype='object')
Validation CSV columns: Index(['Unnamed: 0', 'original_path', 'id', 'label', 'label_str', 'path',
       'full_path'],
      dtype='object')
Test CSV columns: Index(['Unnamed: 0', 'original_path', 'id', 'label', 'label_str', 'path',
       'full_path'],
      dtype='object')


In [ ]:
 from torch.utils.data import Dataset
 from PIL import Image

 class DeepFakeDataset(Dataset):
     def __init__(self, dataframe, transform=None):
         self.df = dataframe.reset_index(drop=True)
         self.transform = transform

     def __getitem__(self, idx):
         img_path = self.df.loc[idx, 'full_path']  # ✅ Not 'path'
         label = 0 if self.df.loc[idx, 'label_str'].lower() == 'real' else 1
         image = Image.open(img_path).convert("RGB")
         if self.transform:
             image = self.transform(image)
         return image, label



In [ ]:
 import torch
 import torch.nn as nn
 import torchvision.models as models

 # Use GPU if available
 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 print("Using device:", device)

 # Load pretrained ResNet18
 model = models.resnet18(pretrained=True)

 # Modify last layer for binary classification (real vs fake)
 model.fc = nn.Linear(model.fc.in_features, 2)
 model = model.to(device)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 203MB/s]


In [ ]:
 from torch.utils.data import Dataset
 from PIL import Image

 class DeepFakeDataset(Dataset):
     def __init__(self, dataframe, transform=None):
         self.df = dataframe.reset_index(drop=True)
         self.transform = transform

     def __len__(self): #  ✅ Required by DataLoader
         return len(self.df)

     def __getitem__(self, idx):
         img_path = self.df.loc[idx, 'full_path']
         label = 0 if self.df.loc[idx, 'label_str'].lower() == 'real' else 1
         image = Image.open(img_path).convert("RGB")
         if self.transform:
             image = self.transform(image)
         return image, label


In [ ]:
#  Create datasets
train_data = DeepFakeDataset(train_df, transform=transform)
val_data = DeepFakeDataset(val_df, transform=transform)
test_data = DeepFakeDataset(test_df, transform=transform)

 # Create dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=32, num_workers=2, pin_memory=True)


print("✅ Data is now ready for model training.")


NameError: name 'transform' is not defined

In [ ]:
 import torch.optim as optim
 import torch.nn as nn

 criterion = nn.CrossEntropyLoss()
 optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
 def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
     train_acc, val_acc = [], []
     train_loss, val_loss = [], []

     for epoch in range(num_epochs):
         model.train()
         running_loss, correct, total = 0, 0, 0

         for images, labels in train_loader:
             images, labels = images.to(device), labels.to(device)

             optimizer.zero_grad()
             outputs = model(images)
             loss = criterion(outputs, labels)
             loss.backward()
             optimizer.step()

             running_loss += loss.item()
             _, predicted = torch.max(outputs.data, 1)
             total += labels.size(0)
             correct += (predicted == labels).sum().item()

         train_accuracy = 100 * correct / total
         train_acc.append(train_accuracy)
         train_loss.append(running_loss / len(train_loader))

          #Validation phase
         model.eval()
         val_running_loss, val_correct, val_total = 0, 0, 0

         with torch.no_grad():
             for images, labels in val_loader:
                 images, labels = images.to(device), labels.to(device)
                 outputs = model(images)
                 loss = criterion(outputs, labels)
                 val_running_loss += loss.item()
                 _, predicted = torch.max(outputs.data, 1)
                 val_total += labels.size(0)
                 val_correct += (predicted == labels).sum().item()

         val_accuracy = 100 * val_correct / val_total
         val_acc.append(val_accuracy)
         val_loss.append(val_running_loss / len(val_loader))

         print(f"Epoch {epoch+1}/{num_epochs} | "
               f"Train Acc: {train_accuracy:.2f}% | Val Acc: {val_accuracy:.2f}%")

     return train_acc, val_acc, train_loss, val_loss


In [ ]:
 train_acc, val_acc, train_loss, val_loss = train_model(
     model, train_loader, val_loader, criterion, optimizer, num_epochs=5
 )


NameError: name 'train_loader' is not defined

In [ ]:


 import kagglehub
 import os
 import pandas as pd
 from PIL import Image
 from tqdm import tqdm

 import torch
 import torch.nn as nn
 import torch.optim as optim
 from torchvision import transforms, models
 from torch.utils.data import Dataset, DataLoader


 full_path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")
 print("✅ Dataset downloaded to:", full_path)


 base_path = full_path  # Already downloaded
 image_root = os.path.join(base_path, "real_vs_fake", "real-vs-fake")

 train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
 val_df = pd.read_csv(os.path.join(base_path, "valid.csv"))
 test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

 # Add full image paths
 for df in [train_df, val_df, test_df]:
     df['full_path'] = df['path'].apply(lambda x: os.path.join(image_root, x))

 print("✅ CSVs loaded | Train:", len(train_df), "| Val:", len(val_df), "| Test:", len(test_df))

 class DeepFakeDataset(Dataset):
     def __init__(self, dataframe, transform=None):
         self.df = dataframe.reset_index(drop=True)
         self.transform = transform

     def __len__(self):
         return len(self.df)

     def __getitem__(self, idx):
         img_path = self.df.loc[idx, 'full_path']
         label = 0 if self.df.loc[idx, 'label_str'].lower() == 'real' else 1
         image = Image.open(img_path).convert("RGB")
         if self.transform:
             image = self.transform(image)
         return image, label

 transform = transforms.Compose([
     transforms.Resize((128, 128)),
     transforms.ToTensor()
 ])

 # 🧪 Use small subset first (for speed testing)
 train_df_small = train_df.sample(n=1000, random_state=42)
 val_df_small = val_df.sample(n=200, random_state=42)

 train_data = DeepFakeDataset(train_df_small, transform=transform)
 val_data = DeepFakeDataset(val_df_small, transform=transform)

 train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
 val_loader = DataLoader(val_data, batch_size=32, num_workers=2, pin_memory=True)

 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 print("🚀 Using device:", device)

 model = models.resnet18(pretrained=True)
 model.fc = nn.Linear(model.fc.in_features, 2)
 model = model.to(device)

 criterion = nn.CrossEntropyLoss()
 optimizer = optim.Adam(model.parameters(), lr=0.0001)

 def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=2):
     train_acc, val_acc = [], []
     train_loss, val_loss = [], []

     for epoch in range(num_epochs):
         model.train()
         running_loss, correct, total = 0, 0, 0

         for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
             images, labels = images.to(device), labels.to(device)
             optimizer.zero_grad()
             outputs = model(images)
             loss = criterion(outputs, labels)
             loss.backward()
             optimizer.step()

             running_loss += loss.item()
             _, predicted = torch.max(outputs.data, 1)
             total += labels.size(0)
             correct += (predicted == labels).sum().item()

         train_accuracy = 100 * correct / total
         train_acc.append(train_accuracy)
         train_loss.append(running_loss / len(train_loader))

         model.eval()
         val_running_loss, val_correct, val_total = 0, 0, 0
         for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
             images, labels = images.to(device), labels.to(device)
             with torch.no_grad():
                 outputs = model(images)
                 loss = criterion(outputs, labels)
                 val_running_loss += loss.item()
                 _, predicted = torch.max(outputs.data, 1)
                 val_total += labels.size(0)
                 val_correct += (predicted == labels).sum().item()

         val_accuracy = 100 * val_correct / val_total
         val_acc.append(val_accuracy)
         val_loss.append(val_running_loss / len(val_loader))

         print(f"✅ Epoch {epoch+1}: Train Acc = {train_accuracy:.2f}%, Val Acc = {val_accuracy:.2f}%")

     return train_acc, val_acc, train_loss, val_loss


 train_acc, val_acc, train_loss, val_loss = train_model(
     model, train_loader, val_loader, criterion, optimizer, num_epochs=2
 )


✅ Dataset downloaded to: /kaggle/input/140k-real-and-fake-faces
✅ CSVs loaded | Train: 100000 | Val: 20000 | Test: 20000
🚀 Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/2 - Validation: 100%|██████████| 7/7 [00:00<00:00,  7.52it/s]


✅ Epoch 1: Train Acc = 69.90%, Val Acc = 66.00%


Epoch 2/2 - Validation: 100%|██████████| 7/7 [00:00<00:00,  8.52it/s]

✅ Epoch 2: Train Acc = 97.30%, Val Acc = 71.00%


In [ ]:


import kagglehub
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader


full_path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")
print("✅ Dataset downloaded to:", full_path)


base_path = full_path
image_root = os.path.join(base_path, "real_vs_fake", "real-vs-fake")

train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
val_df = pd.read_csv(os.path.join(base_path, "valid.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

for df in [train_df, val_df, test_df]:
    df['full_path'] = df['path'].apply(lambda x: os.path.join(image_root, x))

print(f"✅ CSVs loaded | Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")


class DeepFakeDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'full_path']
        label = 0 if self.df.loc[idx, 'label_str'].lower() == 'real' else 1
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Use more data for training
train_df_small = train_df.sample(n=10000, random_state=42)
val_df_small = val_df.sample(n=2000, random_state=42)

train_data = DeepFakeDataset(train_df_small, transform=transform)
val_data = DeepFakeDataset(val_df_small, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, num_workers=2, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Using device:", device)

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_acc, val_acc = [], []
    train_loss, val_loss = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        train_acc.append(train_accuracy)
        train_loss.append(running_loss / len(train_loader))

        model.eval()
        val_running_loss, val_correct, val_total = 0, 0, 0
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            with torch.no_grad():
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        val_acc.append(val_accuracy)
        val_loss.append(val_running_loss / len(val_loader))

        print(f"✅ Epoch {epoch+1}: Train Acc = {train_accuracy:.2f}%, Val Acc = {val_accuracy:.2f}%")

    return train_acc, val_acc, train_loss, val_loss

# Start training
train_acc, val_acc, train_loss, val_loss = train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=5
)


test_data = DeepFakeDataset(test_df.sample(n=2000), transform=transform)
test_loader = DataLoader(test_data, batch_size=32, num_workers=2, pin_memory=True)

def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"📊 Test Accuracy: {acc:.2f}%")

evaluate_model(model, test_loader)


plt.figure(figsize=(8, 4))
plt.plot(train_acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.show()


100%|██████████| 3.75G/3.75G [00:43<00:00, 92.5MB/s]

Extracting files...


✅ Dataset downloaded to: /root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2
✅ CSVs loaded | Train: 100000 | Val: 20000 | Test: 20000
🚀 Using device: cpu


Epoch 1/5 - Validation: 100%|██████████| 63/63 [01:13<00:00,  1.17s/it]


✅ Epoch 1: Train Acc = 77.72%, Val Acc = 85.10%


...

Epoch 3/5 - Validation: 100%|██████████| 63/63 [01:20<00:00,  1.27s/it]


✅ Epoch 3: Train Acc = 91.76%, Val Acc = 88.85%


Epoch 4/5 - Validation: 100%|██████████| 63/63 [01:13<00:00,  1.17s/it]


✅ Epoch 4: Train Acc = 93.62%, Val Acc = 90.40%


Epoch 5/5 - Training:  73%|███████▎  | 227/313 [12:38<04:46,  3.34s/it]

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving download.jpeg to download.jpeg


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

 Use same transform as training
transform_eval = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

label_map = {0: 'Real', 1: 'Fake'}

def predict_and_display(image_path):
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform_eval(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=1)
        conf_score, pred_class = torch.max(probs, dim=1)

    predicted_label = label_map[pred_class.item()]
    confidence = conf_score.item() * 100

     Show image and prediction
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{predicted_label} ({confidence:.2f}%)", fontsize=14)
    plt.show()

    print(f"📸 {os.path.basename(image_path)} → {predicted_label} ({confidence:.2f}%)")

 Predict for each uploaded image
for fname in uploaded.keys():
    predict_and_display(fname)
